In [2]:
# The following 5 code cells download the dataset from kaggle
!pip install -q kaggle


In [3]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anirudhragam19","key":"f5750430c16e8587588e2a5569998b0e"}'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [5]:
! kaggle datasets list


ref                                                          title                                           size  lastUpdated          downloadCount  
-----------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  
alexgude/california-traffic-collision-data-from-switrs       California Traffic Collision Data from SWITRS    1GB  2020-11-22 16:51:55            188  
babyoda/women-entrepreneurship-and-labor-force               Women Entrepreneurship and Labor Force           1KB  2020-11-21 08:38:51           1078  
szymonjanowski/internet-articles-data-with-users-engagement  Internet news data with readers engagement       3MB  2020-11-21 17:09:57            443  
sakshigoyal7/credit-card-customers                           Credit Card customers                          379KB  2020-11-19 07:38:44           1868  
imoore/2020-us-general-election-turnout-rates                2020 US General Election Tu

In [6]:
! mkdir traffic_dataset
! kaggle datasets download -d silicon99/dft-accident-data
! unzip dft-accident-data.zip -d traffic_dataset/


 87% 97.0M/112M [00:01<00:00, 56.0MB/s]
100% 112M/112M [00:01<00:00, 77.0MB/s] 
Archive:  dft-accident-data.zip
  inflating: traffic_dataset/Accidents0515.csv  
  inflating: traffic_dataset/Casualties0515.csv  
  inflating: traffic_dataset/Vehicles0515.csv  
  inflating: traffic_dataset/contextCSVs/Accident_Severity.csv  
  inflating: traffic_dataset/contextCSVs/Age_Band.csv  
  inflating: traffic_dataset/contextCSVs/Casualty_Class.csv  
  inflating: traffic_dataset/contextCSVs/Casualty_Severity.csv  
  inflating: traffic_dataset/contextCSVs/Casualty_Type.csv  
  inflating: traffic_dataset/contextCSVs/Day_of_Week.csv  
  inflating: traffic_dataset/contextCSVs/Journey_Purpose.csv  
  inflating: traffic_dataset/contextCSVs/Junction_Control.csv  
  inflating: traffic_dataset/contextCSVs/Junction_Detail.csv  
  inflating: traffic_dataset/contextCSVs/Junction_Location.csv  
  inflating: traffic_dataset/contextCSVs/Light_Conditions.csv  
  inflating: traffic_dataset/contextCSVs/Local_Authori

In [7]:
#Importing the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

#Reading the csv file into a pandas dataframe
accidents = pd.read_csv('traffic_dataset/Accidents0515.csv')

#Displaying the first 5 rows of the dataframe
accidents.head()



,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
0,200501BS00001,525680.0,178240.0,-0.191170,51.489096,1,2,1,1,04/01/2005,3,17:42,12,E09000020,3,3218,6,30,0,-1,-1,0,0,1,1,2,2,0,0,1,1,E01002849
1,200501BS00002,524170.0,181650.0,-0.211708,51.520075,1,3,1,1,05/01/2005,4,17:36,12,E09000020,4,450,3,30,6,2,5,0,0,5,4,1,1,0,0,1,1,E01002909
2,200501BS00003,524520.0,182240.0,-0.206458,51.525301,1,3,2,1,06/01/2005,5,00:15,12,E09000020,5,0,6,30,0,-1,-1,0,0,0,4,1,1,0,0,1,1,E01002857
3,200501BS00004,526900.0,177530.0,-0.173862,51.482442,1,3,1,1,07/01/2005,6,10:35,12,E09000020,3,3220,6,30,0,-1,-1,0,0,0,1,1,1,0,0,1,1,E01002840
4,200501BS00005,528060.0,179040.0,-0.156618,51.495752,1,3,1,1,10/01/2005,2,21:13,12,E09000020,6,0,6,30,0,-1,-1,0,0,0,7,1,2,0,0,1,1,E01002863


In [8]:
# cols contains all the columns in the dataset that we are using to train the model, including the target variable i.e. Accident Severity
cols = ['Longitude','Latitude','1st_Road_Number','Day_of_Week','Light_Conditions','Weather_Conditions','Speed_limit','Number_of_Vehicles','Road_Surface_Conditions','Accident_Severity']
acc = accidents[cols]

#Dropping rows with missing alues
acc.dropna(inplace=True)

# Splitting the dataset into the X's and y's of the model i.e. the inputs to the model and the outputs
X = acc[cols[:-1]]
y = acc[cols[-1]]

# Splitting data into train and test sets. We're using a 80-20 split of the dataset 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) 






/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Creating the decision tree classifier 
clf = DecisionTreeClassifier()

# Fitting the decision tree with the train set
clf = clf.fit(X_train,y_train)

#Predicting on the test set
y_pred = clf.predict(X_test)

print('Accuracy: ',accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))





In [13]:
#Bagging with 10 decision tree classifiers with Bootstrapping
from sklearn.ensemble import BaggingClassifier 

dt = DecisionTreeClassifier() 

#Creating the bagging model instance
model = BaggingClassifier(base_estimator = dt, 
                          n_estimators = 10, 
                          bootstrap=True) 

#Fitting the model on the training set
model = model.fit(X_train,y_train)

#Predicting on the test set
y_pred1 = model.predict(X_test)

print('Accuracy: ',accuracy_score(y_test,y_pred1))
print(classification_report(y_test,y_pred1))




In [10]:
# Bagging with 10 decision tree classifiers without Bootstrapping

#Creating the bagging model instance
model = BaggingClassifier(base_estimator = dt, 
                          n_estimators = 10, 
                          bootstrap=False) 

#Fitting the model on the training set
model = model.fit(X_train,y_train)

#Predicting on the test set
y_pred2 = model.predict(X_test)

print('Accuracy: ',accuracy_score(y_test,y_pred2))
print(classification_report(y_test,y_pred2))



Accuracy:  0.743124882407618
              precision    recall  f1-score   support

           1       0.03      0.04      0.03      4553
           2       0.17      0.19      0.18     48299
           3       0.86      0.84      0.85    303251

    accuracy                           0.74    356103
   macro avg       0.35      0.36      0.35    356103
weighted avg       0.76      0.74      0.75    356103



In [11]:
# Bagging with 50 decision tree classifiers with Bootstrapping

#Creating the bagging model instance
model = BaggingClassifier(base_estimator = dt, 
                          n_estimators = 50, 
                          bootstrap=True) 

#Fitting the model on the training set
model = model.fit(X_train,y_train)

#Predicting on the test set
y_pred2 = model.predict(X_test)

print('Accuracy: ',accuracy_score(y_test,y_pred2))
print(classification_report(y_test,y_pred2))



Accuracy:  0.8268983973737936
              precision    recall  f1-score   support

           1       0.06      0.01      0.02      4553
           2       0.21      0.06      0.10     48299
           3       0.86      0.96      0.91    303251

    accuracy                           0.83    356103
   macro avg       0.37      0.34      0.34    356103
weighted avg       0.76      0.83      0.78    356103



In [14]:
# Bagging with 100 decision tree classifiers with Bootstrapping

#Creating the bagging model instance
model = BaggingClassifier(base_estimator = dt, 
                          n_estimators = 100, 
                          bootstrap=True) 

#Fitting the model on the training set
model = model.fit(X_train,y_train)

#Predicting on the test set
y_pred2 = model.predict(X_test)

print('Accuracy: ',accuracy_score(y_test,y_pred2))
print(classification_report(y_test,y_pred2))

Accuracy:  0.8289090515946229
              precision    recall  f1-score   support

           1       0.06      0.01      0.01      4553
           2       0.21      0.06      0.09     48299
           3       0.86      0.96      0.91    303251

    accuracy                           0.83    356103
   macro avg       0.37      0.34      0.34    356103
weighted avg       0.76      0.83      0.78    356103

